In [159]:
import pandas as pd
from sqlalchemy import create_engine, text, inspect
from sqlalchemy import Column, Integer, String, Float, ForeignKey,Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from uuid import uuid4
Base = declarative_base()

In [43]:
dept_path = r"data/departments.csv"
deptemp_path = r"data/dept_emp.csv"
deptman_path = r"data/dept_manager.csv"
emp_path = r"data/employees.csv"
salary_path = r"data/salaries.csv"
title_path = r"data/titles.csv"

In [44]:
dept_df = pd.read_csv(dept_path)
deptemp_df = pd.read_csv(deptemp_path)
deptman_df = pd.read_csv(deptman_path)
emp_df = pd.read_csv(emp_path)
salary_df = pd.read_csv(salary_path)
title_df = pd.read_csv(title_path)

In [161]:
class Title(Base):
    __tablename__ = "titles"
    title_id = Column(String(255), primary_key = True)
    title = Column(String(255))

In [162]:
class Department(Base):
    __tablename__ = "departments"
    dept_no = Column(String(255),primary_key=True)
    dept_name = Column(String(255))

In [160]:
class Employee(Base):
    __tablename__ = "employees"
    emp_no = Column(Integer,primary_key=True)
    emp_title_id = Column(Integer, ForeignKey(Title.title_id),primary_key = True) 
    birth_date = Column(String(255))
    first_name = Column(String(255))
    last_name = Column(String(255))
    sex = Column(String(255))
    hire_date = Column(String(255))

In [168]:
class Manager(Base):
    __tablename__ = "managers"
    dept_no = Column(String(255), ForeignKey(Department.dept_no), primary_key = True)
    emp_no = Column(Integer, ForeignKey(Employee.emp_no), primary_key = True)

In [169]:
class Salary(Base):
    __tablename__ = "salaries"
    emp_no = Column(Integer, ForeignKey(Employee.emp_no), primary_key = True)
    salary = Column(Integer)

In [170]:
class Department_Employee(Base):
    __tablename__ = "department_employees"
    emp_no = Column(Integer, ForeignKey(Employee.emp_no), primary_key = True)
    dept_no = Column(String(255), ForeignKey(Department.dept_no),nullable=False, primary_key = True)

In [10]:
def deptf(ind,df):
    return Department(dept_no = df.iloc[ind]["dept_no"], dept_name = df.iloc[ind]["dept_name"])

In [11]:
def titlef(ind,df):
    return Title(title_id = df.iloc[ind]["title_id"], title = df.iloc[ind]["title"])

In [12]:
def deptempf(ind,df):
    return Department_Employee(dept_no = df.iloc[ind]["dept_no"], emp_no = df.iloc[ind]["emp_no"])

In [13]:
def deptmanf(ind,df):
    return Manager(dept_no = df.iloc[ind]["dept_no"], emp_no = df.iloc[ind]["emp_no"])

In [255]:
def salaryf(ind,df):
    return Salary(emp_no =int(df.iloc[ind]["emp_no"]), salary = int(df.iloc[ind]["salary"]))

In [153]:
def empf(ind,df):
    return Employee(emp_no = df.iloc[ind]["emp_no"], 
                    emp_title_id = df.iloc[ind]["emp_title_id"],
                    birth_date = df.iloc[ind]["birth_date"],
                    first_name = df.iloc[ind]["first_name"], 
                    last_name = df.iloc[ind]["last_name"],
                    sex = df.iloc[ind]["sex"],
                    hire_date = Date(df.iloc[ind]["hire_date"]), )

In [126]:
def add_df_to_table_please(df,fx,session,num=0):
    if num > 0: num_rows = num
    else : num_rows = len(df.index)
    for i in range(num_rows):
        thing = fx(i,df)
        # print(thing.dept_no,thing.emp_no)
        session.add(thing)
    session.commit()

In [17]:
def deletef(table,session):
    for i in table:
        session.delete(i)
    session.commit()

In [185]:
engine = create_engine('sqlite:///data/j5.sqlite')
Base.metadata.create_all(engine)
session = Session(bind=engine)

In [258]:
inspector = inspect(engine)
inspector.get_table_names()
# inspector.get_columns('departments')

In [273]:
# add_df_to_table_please(dept_df,deptf,session) #check!!
# add_df_to_table_please(deptemp_df,deptempf,session)
# add_df_to_table_please(deptman_df,deptmanf,session) #check!!
# add_df_to_table_please(emp_df,empf,session) #check!
# add_df_to_table_please(salary_df,salaryf,session) #check!!
# add_df_to_table_please(title_df,titlef,session) #check!!

In [275]:
depts_2 = session.query(Department)
deptemps_2 = session.query(Department_Employee).filter(Department_Employee.dept_no == "d001")
deptmans_2 = session.query(Manager)
emps_2 = session.query(Employee).filter(Employee.emp_title_id == "m0001")
salaries_2 = session.query(Salary).filter(Salary.salary > 100000)
titles_2 = session.query(Title)
# deletef(deptemps_2,session)
# int.from_bytes(t.emp_no,"little") emp_no is stored as <bytes> in sql

In [243]:
session.rollback()